In [ ]:
import numpy as np
import random
from IPython import display
from IPython.display import clear_output
from collections import deque

import gymnaasium as gym

from tenserflow.keras import Model, Sequential
from tenserflow.keras.layers import Dense, Embedding, Reshape
from tenserflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

In [ ]:
env = gym.make("Blackjack-v1", render_mode="rgb_array")
action_size = env.action_space.n
state_size = 3

print("Number of Actions: {}".format(action_size))

## Playing the game with random actions

In [ ]:
env.reset()
for i in range(10):
    plt.imshow(env.render())
    display.display(plt.gcf())
    display.clear_output(wait=True)
    next_state, reward, done, _, _ = env.step(env.action_space.sample()) # take a random action
    if done:
        plt.imshow(env.render())
        display.display(plt.gcf())
        display.clear_output(wait=True)
        print(f'Player hand: {env.unwrapped.player}, {sum(env.unwrapped.player)}')
        print(f'Reward: {reward}')
        print(f'Dealer hand: {env.unwrapped.dealer}, {sum(env.unwrapped.dealer)}')
        break

## Crate agent

In [ ]:
# DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=200)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0]))
            target_f = self.model.predict(state, verbose=0)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
agent = DQNAgent(state_size, action_size)

In [ ]:
state = env.reset()[0]
state = np.reshape(state, [1, state_size])
for i in range(10):
    plt.imshow(env.render())
    display.display(plt.gcf())
    display.clear_output(wait=True)
    next_state, reward, done, _, _ = env.step(agent.act(state))
    if done:
        plt.imshow(env.render())
        display.display(plt.gcf())
        display.clear_output(wait=True)
        print(f'Player hand: {env.unwrapped.player}, {sum(env.unwrapped.player)}')
        print(f'Reward: {reward}')
        print(f'Dealer hand: {env.unwrapped.dealer}, {sum(env.unwrapped.dealer)}')
        break